In [10]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import math
import pyproj
import descartes
from descartes import PolygonPatch
import geopandas as gpd 
import shapely.geometry as geometry
import shapefile 
from functools import partial 
import shapely.ops as ops 
import fiona

In [11]:
pw = pd.read_parquet('part-00000-34c001ad-b3a5-4bab-abe9-e3d1be553f75-c000.gz.parquet')

In [12]:
pw = pw.rename(columns={'a_location.location_latitude' : 'latitude', 'a_location.location_longitude' : 'longitude'})
pw.head()

In [13]:
locations = pw[['outage_time', 'latitude', 'longitude', 'cluster_size']]
area_locations = locations[locations['cluster_size'] > 3]
cluster_size_is_2 = locations[locations['cluster_size'] == 2]
location_838 = locations[locations['outage_time'] == 1531651143]
location_0 = locations[locations['outage_time'] == 1530813017]

In [14]:
street_map = gpd.read_file('/Users/emilypaszkiewicz17/Desktop/research/map_w_coordinates/GHA-4_admin_SHP/GHA-4.shp')
fig,ax = plt.subplots(figsize = (15,15))
street_map.plot(ax=ax)

In [15]:
plt.scatter(location_838['longitude'], location_838['latitude'])

In [9]:
location_838

In [9]:
#Create points out of the lat and long data 
crs = {'init', 'epsg:4326'}
geom = [geometry.Point(xy) for xy in zip(location_838['longitude'], location_838['latitude'])]
geo_location_838 = gpd.GeoDataFrame(location_838, crs=crs, geometry=geom)
geo_location_838.to_csv(path_or_buf='/Users/emilypaszkiewicz17/Desktop/research/geo_location_838.csv')
geo_location_838.head()


In [10]:
fig,ax = plt.subplots(figsize = (20,20))
street_map.plot(ax=ax)
geo_location_838['geometry'].plot(ax=ax, color='red')
plt.title('Outages at outage time 1531651143')
#generated using https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972

In [11]:
crs = {'init', 'epsg:4326'}
geom = [geometry.Point(xy) for xy in zip(location_0['longitude'], location_0['latitude'])]
geo_location_0 = gpd.GeoDataFrame(location_0, crs=crs, geometry=geom)
fig,ax = plt.subplots(figsize = (20,20))
street_map.plot(ax=ax)
geo_location_0['geometry'].plot(ax=ax, color='red')
plt.title('Outages at outage time 1530813017')

In [12]:
#concave_area takes in two lists or arrays and outputs the concave area of the polygon in Euclidian distance using the crossproduct  
def concave_area(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

In [14]:
#convert the area_locations dataframe to a dataframe where the locations are listed and indexed by the outage_time 
list_locations = area_locations.groupby('outage_time')['latitude'].apply(lambda x: x.to_list()).reset_index()
list_locations['longitude'] = area_locations.groupby('outage_time')['longitude'].apply(lambda x: x.to_list()).values
list_locations['cluster_size'] = area_locations.groupby('outage_time')['cluster_size'].first().values
list_locations = list_locations.set_index('outage_time')
#now apply the area function to the latitudes and longitudes
list_locations['concave_area'] = (np.vectorize(concave_area)(list_locations['longitude'], list_locations['latitude']))
list_locations.head()

In [15]:
#Creating a Polygon class for each of the clusters 
check = []
for i in range(len(list_locations)):
    a = list_locations.iloc[i, :].values[0]
    b = list_locations.iloc[i, :].values[1]
    check.append(list(zip(a, b)))
list_locations['poly'] = check
poly = [geometry.Polygon(x, holes=None) for x in list_locations['poly']]
geo_location_poly = gpd.GeoDataFrame(list_locations, crs=crs, geometry=poly)

geo_location_poly['convex_area'] = geo_location_poly.convex_hull.area
geo_location_poly.head()

In [16]:
#calculating area with the polygon class 
#ops.transform is transforming geom by first argument (aka everything following partial). Partial creates a partial object 
#https://gis.stackexchange.com/questions/127607/area-in-km-from-polygon-of-coordinates 

def convex_area(a_geom): 
    return ops.transform(partial(
        pyproj.transform, 
        pyproj.Proj(init='EPSG:4326'), 
        pyproj.Proj(proj='aea', lat1=a_geom.bounds[0], long1=a_geom.bounds[1], lat2=a_geom.bounds[2], long2=a_geom.bounds[3])), a_geom).convex_hull.area

geo_location_poly['convex_area_m^2'] = (np.vectorize(convex_area)(geo_location_poly['geometry']))
geo_location_poly.head()

In [18]:
#now let's do the same thing for coordinates, but projected onto EPSG:32630 

wgs84 = pyproj.Proj('+init=EPSG:4326')
wgs84_tz30 = pyproj.Proj('+init=EPSG:32630')
# ghana_poly = pyproj.transform(wgs84, wgs84_tz30)

def convex_area_ghana(a_geom): 
    return ops.transform(partial(pyproj.transform, wgs84, wgs84_tz30), a_geom).convex_hull.area

geo_location_poly['ghana_area'] = (np.vectorize(convex_area_ghana)(geo_location_poly['geometry']))
geo_location_poly.head()

In [19]:
#now let's compare the areas computed with this algorithm vs. areas in qGIS 
#we will examine the areas computed for outage_time 1531651143 
#note that these were computed in different projections, however from what I have heard, these projections shouldn't make that much of a difference since ESPG:32630 is near the equator 
#python areas were computed with Albers Equal Area projection 
#qgis areas were computed with an EPSG:32630 projection 
aea_degrees = geo_location_poly[geo_location_poly.index == 1531651143]['convex_area'].values[0]
aea_meters = geo_location_poly[geo_location_poly.index == 1531651143]['convex_area_m^2'].values[0]
ghana_meters = geo_location_poly[geo_location_poly.index == 1531651143]['ghana_area'].values[0]
qgis_ghana_degrees = 0.004044263111149915
qgis_ghana_meters = 49546874.7356604
degree_squared_dif = abs(aea_degrees - qgis_ghana_degrees)
meters_squared_dif = abs(aea_meters - qgis_ghana_meters)
ghana_meters_squared_dif = abs(ghana_meters - qgis_ghana_meters)
meters_squared_dif, ghana_meters_squared_dif


Ok we have a little bit of a problem here. The difference in areas between the Albers projection and the EPSG:32630 in qgis is almost 6 times smaller than the difference in areas between the EPSG:32630 projection in python and in qgis!!!
So wtf does that even MEAN?? 
It is possible I implemented the projection incorrectly(although the values are not like crazy off so that's not super likely) or there is some other problem that I can't think of?? WTF do I doooooo
Perhaps this article will help: https://all-geo.org/volcan01010/2012/11/change-coordinates-with-pyproj/ 

In [20]:
#plot the cluster size against the area covered by coordinates (Euclidian distance)
#consider plotting on a log scale 
plt.scatter(list_locations['cluster_size'], list_locations['concave_area'])
plt.xlabel('Cluster Size')
plt.ylabel('Area Covered by Coordinates (deg)')
plt.title('Cluster Size vs. Area Covered by Coordinates (deg)')
plt.ylim(0, 0.0025)

In [21]:
plt.scatter(list_locations['cluster_size'], (list_locations['convex_area_m^2']/1000000))
plt.xlabel('Cluster Size')
plt.ylabel('Area Covered by Coordinates (m^2)')
plt.title('Cluster Size vs. Area Covered by Coordinates (m^2)')


In [22]:
def distance(latitude, longitude): 
    return np.sqrt((longitude[1]-longitude[0])**2 + (latitude[1]-latitude[0])**2)

In [23]:
def coordinate_distance(latitude, longitude): 
    a1 = latitude[0]
    a2 = latitude[1]
    b1 = longitude[0]
    b2 = longitude[1]
    rad = 6371 #km 
    num = np.arccos(np.cos(a1)*np.cos(b1)*np.cos(a2)*np.cos(b2) + np.cos(a1)*np.sin(b1)*np.cos(a2)*np.sin(b2) + np.sin(a1)*np.sin(a2))
    denom = 360*2*np.pi*rad
    return num/denom 
# computed in km since rad is the radius of the earth in km
# http://mathforum.org/library/drmath/view/51711.html <-- formula used can be found here 

In [24]:
#create a dataframe for points with cluster size 2 
two_locations = cluster_size_is_2.groupby('outage_time')['latitude'].apply(lambda x: x.to_list()).reset_index()
two_locations['longitude'] = cluster_size_is_2.groupby('outage_time')['longitude'].apply(lambda x: x.to_list()).values
two_locations['cluster_size'] = cluster_size_is_2.groupby('outage_time')['cluster_size'].first().values
two_locations = two_locations.set_index('outage_time')
two_locations['distance'] = (np.vectorize(distance)(two_locations['latitude'], two_locations['longitude']))
two_locations['coordinate_distance (km)'] = (np.vectorize(coordinate_distance)(two_locations['latitude'], two_locations['longitude']))
two_locations.head()

In [25]:
plt.hist(two_locations['coordinate_distance (km)'], bins=20)
plt.title('Distance Between Two Clusters During a Power Outage (km)')
plt.xlabel('')

In [26]:
plt.hist(two_locations['distance'], bins=20)
plt.title('Distance Between Two Clusters During a Power Outage (deg)')

In [ ]:
#let's save this to a csv so that we can use this data in qGIS 
geo_location_poly.to_csv(path_or_buf='/Users/emilypaszkiewicz17/Desktop/research/geo_location_poly.csv')

In [ ]:
#try implementing code from http://blog.thehumangeo.com/2014/05/12/drawing-boundaries-in-python/
#to do so, take one cluster and convert points into a multipoint list. Than run plot_polygon on that list 

# multi_point = [geometry.MultiPoint(x) for x in list_locations['poly']]
# geo_location_multi_point = gpd.GeoDataFrame(list_locations, crs=crs, geometry=multi_point)
# geo_location_multi_point['convexed'] = [multi_point[i].convex_hull for i in range(len(multi_point))]
# geo_location_multi_point.head() 
# geo_location_multi_point['convexed'].values[0]


def plot_polygon(polygon): 
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    margin = .3 
    x_min, y_min, x_max, y_max = polygon.bounds
    ax.set_xlim([x_min-margin, x_max+margin])
    ax.set_ylim([y_min-margin, y_max-margin])
    patch = PolygonPatch(polygon, fc='#999999',
                         ec='#000000', fill=True,
                         zorder=-1)
    ax.add_patch(patch)
    return fig

In [ ]:
#this function is a more expensive version of concave_area 

#The function area takes in two lists: latitude and longitude and computes the Euclidian area that they cover 
def area(latitudes, longitudes):
    n = len(latitudes)
    area = 0.0
    for i in range(n):
        j = (i+1)%n
        area += latitudes[i]*longitudes[j]
        area -= latitudes[j]*longitudes[i]
    area = abs(area)/2
    return area 



In [ ]:
#this does not work!!! 
def concave_km_area(lst): 
    rad = 6371 #km 
    cos_term = 1
    sin_term = 1
    cos_and_sin_term = 1
    for lat, long in lst: 
        cos_term = cos_term*np.cos(lat)*np.cos(long)
        sin_term = sin_term*np.sin(lat)
        cos_and_sin_term = cos_and_sin_term*np.cos(lat)*np.sin(long)
    numerator = np.arccos(cos_term + cos_and_sin_term + sin_term)
    denominator = 260*2*np.pi*rad
    return numerator/denominator 

concave_km_area(geo_location_poly['poly'].values[0])
    
    
    
#     a1 = latitude[0]
#     a2 = latitude[1]
#     b1 = longitude[0]
#     b2 = longitude[1]
#     rad = 6371 #km 
#     num = np.arccos(np.cos(a1)*np.cos(b1)*np.cos(a2)*np.cos(b2) + np.cos(a1)*np.sin(b1)*np.cos(a2)*np.sin(b2) + np.sin(a1)*np.sin(a2))
#     denom = 360*2*np.pi*rad
#     return num/denom 